## Advevrsarial Patch On CIFAR10 Using ResNet


In [2]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  print('CUDA is not available.  Training on CPU ...')
else:
  print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


### Prepare dataset

In [3]:
from torchvision import datasets
import torchvision.transforms as transforms
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, Subset

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# convert data to a normalized torch.FloatTensor
print('==> Preparing data..')
#Image augmentation is used to train the model
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
#Only the data is normalaized we do not need to augment the test data
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform_train)
combined_test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform_test)


# Get targets from the test data for stratification
targets = combined_test_data.targets

# Stratified split for validation and test sets from the original test dataset
sss_test_valid = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)  # Split 50/50
valid_idx, test_idx = next(sss_test_valid.split(np.zeros(len(targets)), targets))

# Convert indices to actual Subset for validation and test datasets
valid_dataset = Subset(combined_test_data, valid_idx)
test_dataset = Subset(combined_test_data, test_idx)

# Prepare data loaders
batch_size = 20

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Total number of samples in the complete dataset
total_samples = len(train_loader) + len(valid_loader) + len(test_loader)

# Assuming the splits have been previously calculated:
# Sizes of each dataset split
train_size = len(train_loader)
test_size = len(test_loader)
validation_size = len(valid_loader)

# Calculate proportions
train_proportion = train_size / total_samples
test_proportion = test_size / total_samples
validation_proportion = validation_size / total_samples

# Print the proportions
print(f"Training Data Proportion: {train_proportion:.2f}")
print(f"Test Data Proportion: {test_proportion:.2f}")
print(f"Validation Data Proportion: {validation_proportion:.2f}")


# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

==> Preparing data..
Training Data Proportion: 0.83
Test Data Proportion: 0.08
Validation Data Proportion: 0.08


### Model Definiation

In [4]:
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
  expansion = 1
  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(self.expansion*planes)
      )
  
  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out

class BottleNeck(nn.Module):
  expansion = 4

  def __init__(self, in_planes, planes, stride=1):
    super(BottleNeck, self).__init__()
    self.conv1 = nn.Conv2d(in_planes , planes, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)
    self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm2d(self.expansion*planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes :
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(self.expansion*planes)
      )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = F.relu(self.bn2(self.conv2(out)))
    out = self.bn3(self.conv3(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out

class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_planes = 64

    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512*block.expansion, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion      
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out



In [5]:
ResNet18 = ResNet(BasicBlock, [2,2,2,2])


if train_on_gpu:
  ResNet18 = torch.nn.DataParallel(ResNet18)
  cudnn.benchmark = True

In [6]:

# Load Pretrained model
ResNet18.load_state_dict(torch.load('ResNet18_20250419.pt'))

print("Pretrained ResNet18 loaded and ready!")

# Step 4: Move model to device and eval mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ResNet18 = ResNet18.to(device)
ResNet18.eval()

Pretrained ResNet18 loaded and ready!


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       

### Adversarial Patch

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
     

In [ ]:
# Function to create a random patch
def create_patch(patch_size=(3, 3)):
    patch = torch.randn(3, *patch_size, requires_grad=True, device=device)
    return patch

# Function to apply the patch to images
def place_patch(img, patch):
    batch_size, _, h, w = img.size()
    ph, pw = patch.size(1), patch.size(2)
    for i in range(batch_size):
        x_offset = torch.randint(0, h - ph + 1, (1,)).item()
        y_offset = torch.randint(0, w - pw + 1, (1,)).item()
        img[i, :, x_offset:x_offset+ph, y_offset:y_offset+pw] = patch
    return img

# Training function for adversarial patch
def patch_training_step(model, patch, target_class=None, dataloader=None, optimizer=None, criterion=None):
    model.train()
    total_loss = 0
    for images, _ in dataloader:
        images = images.to(device)
        optimizer.zero_grad()
        patched_images = place_patch(images, patch)
        outputs = model(patched_images)
        if target_class is not None:
            labels = torch.full((images.size(0),), target_class, dtype=torch.long, device=device)
        else:
            labels = torch.randint(0, 10, (images.size(0),), device=device)  # Random class for untargeted attack
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)

# Function to train the adversarial patch
def train_adversarial_patch(model, patch, target_class=None, num_epochs=10):
    patch_optimizer = optim.Adam([patch], lr=0.01)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        loss = patch_training_step(model, patch, target_class, train_loader, patch_optimizer, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

# Function to evaluate the success rate of the adversarial patch
def evaluate_patch(model, patch, dataloader, target_class=None):
    model.eval()
    success = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            patched_images = place_patch(images, patch)
            outputs = model(patched_images)
            _, predicted = outputs.max(1)
            if target_class is not None:
                success += (predicted == target_class).sum().item()
            else:
                success += (predicted != labels).sum().item()  # Evaluate untargeted attack
            total += labels.size(0)

    successR = 100 * success / total
    print(f"Attack Success Rate: {successR:.2f}%")
    return successR


In [ ]:
# CIFAR-10 class names
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Different patch sizes
patch_sizes = [(3, 3), (5, 5), (7, 7), (16, 16)]

ASR_dict = {}

# Train and evaluate patches of different sizes
for patch_size in patch_sizes:
    ASR_dict[patch_size] = list()
    print(f"\nTraining for patch size: {patch_size}")
    for c in range(10):
    
        patch = create_patch(patch_size)
        train_adversarial_patch(ResNet18, patch, target_class=c, num_epochs=20)
        # Save the patch
        torch.save(patch, f'adv_patch_{patch_size[0]}x{patch_size[1]}_targeted_{classes[c]}_20250424.pth')

        # Evaluate attack success rate
        ASR = evaluate_patch(ResNet18, patch, test_loader, target_class=c)
        ASR_dict[patch_size].append(ASR)



Training for patch size: (16, 16)
Epoch 1/20, Loss: 1.6280
Epoch 2/20, Loss: 0.7271
Epoch 3/20, Loss: 0.4865
Epoch 4/20, Loss: 0.3757
Epoch 5/20, Loss: 0.3110
Epoch 6/20, Loss: 0.2641
Epoch 7/20, Loss: 0.2297
Epoch 8/20, Loss: 0.2067
Epoch 9/20, Loss: 0.1920
Epoch 10/20, Loss: 0.1802
Epoch 11/20, Loss: 0.1700
Epoch 12/20, Loss: 0.1612
Epoch 13/20, Loss: 0.1551
Epoch 14/20, Loss: 0.1484
Epoch 15/20, Loss: 0.1451
Epoch 16/20, Loss: 0.1392
Epoch 17/20, Loss: 0.1364
Epoch 18/20, Loss: 0.1322
Epoch 19/20, Loss: 0.1292
Epoch 20/20, Loss: 0.1271
Attack Success Rate: 99.68%
Epoch 1/20, Loss: 3.3206
Epoch 2/20, Loss: 1.9742
Epoch 3/20, Loss: 1.4719
Epoch 4/20, Loss: 1.2258
Epoch 5/20, Loss: 1.0544
Epoch 6/20, Loss: 0.9512
Epoch 7/20, Loss: 0.8742
Epoch 8/20, Loss: 0.8242
Epoch 9/20, Loss: 0.7717
Epoch 10/20, Loss: 0.7375
Epoch 11/20, Loss: 0.7073
Epoch 12/20, Loss: 0.6759
Epoch 13/20, Loss: 0.6526
Epoch 14/20, Loss: 0.6309
Epoch 15/20, Loss: 0.6128
Epoch 16/20, Loss: 0.5928
Epoch 17/20, Loss: 

In [12]:
# CIFAR-10 class names
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Different patch sizes
patch_sizes = [(3, 3), (5, 5), (7, 7), (16, 16)]
ASR_dict = {}
# Evaluate patches of different sizes
for patch_size in patch_sizes:
    ASR_dict[patch_size] = list()
    for c in range(10):
        print(f"\nEvaluating targeted adversarial patch, size: {patch_size}, target class: {classes[c]}")
        patch = create_patch(patch_size)
        patch = torch.load(f'adv_patch_{patch_size[0]}x{patch_size[1]}_targeted_{classes[c]}_20250424.pth')
        # Evaluate attack success rate        
        ASR = evaluate_patch(ResNet18, patch, test_loader, target_class=c)
        ASR_dict[patch_size].append(ASR)
    


Evaluating targeted adversarial patch, size: (3, 3), target class: plane
Attack Success Rate: 53.26%

Evaluating targeted adversarial patch, size: (3, 3), target class: car
Attack Success Rate: 0.00%

Evaluating targeted adversarial patch, size: (3, 3), target class: bird
Attack Success Rate: 6.76%

Evaluating targeted adversarial patch, size: (3, 3), target class: cat
Attack Success Rate: 17.72%

Evaluating targeted adversarial patch, size: (3, 3), target class: deer
Attack Success Rate: 0.00%

Evaluating targeted adversarial patch, size: (3, 3), target class: dog
Attack Success Rate: 0.18%

Evaluating targeted adversarial patch, size: (3, 3), target class: frog
Attack Success Rate: 0.08%

Evaluating targeted adversarial patch, size: (3, 3), target class: horse
Attack Success Rate: 0.00%

Evaluating targeted adversarial patch, size: (3, 3), target class: ship
Attack Success Rate: 6.78%

Evaluating targeted adversarial patch, size: (3, 3), target class: truck
Attack Success Rate: 0.00

In [13]:
Summary_list = []
for s in ASR_dict:
    Summary_list.append([s]+ ASR_dict[s])

In [14]:
import pandas as pd
Summary = pd.DataFrame(Summary_list, columns=['Patch Size'] + classes)
Summary.to_csv('ASR_summary_targeted_20250425.csv', index=False)